In [1]:
from mxnet import autograd,nd
from mxnet.gluon import nn

# 本函数已保存在d2l
# corr2d使⽤了对单个元素赋值（[i, j]=）的操作因而⽆法⾃动求梯度。
def corr2d(X,K):# 输⼊数组X(3x3)与核数组K(2x2)
    h,w=K.shape
    Y=nd.zeros((X.shape[0]-h+1,X.shape[1]-w+1))# 输出数组Y(2x2)
    for i in range(Y.shape[0]):# 卷积窗口按行滚动
        for j in range(Y.shape[1]):# 按列滚动
            Y[i,j]=(X[i:i+h,j:j+w]*K).sum()
    return Y

In [2]:
X=nd.array([[0,1,2],[3,4,5],[6,7,8]])
K=nd.array([[0,1],[2,3]])
corr2d(X,K)


[[19. 25.]
 [37. 43.]]
<NDArray 2x2 @cpu(0)>

In [3]:
# 5.1.2 ⼆维卷积层 
# ⼆维卷积层将输⼊和卷积核做互相关运算，并加上⼀个标量偏差来得到输出。卷积层的模型参数包括了卷积核和标量偏差。
# 在训练模型的时候，通常我们先对卷积核随机初始化，然后不断迭代卷积核和偏差。
class Conv2D(nn.Block):
    def __init__(self,kernel_size,**kwargs):
        super(Conv2D,self).__init__(**kwargs)
        self.weight=self.params.get('weight',shape=kernel_size)# 卷积核
        self.bias=self.params.get('bias',shape=(1,))# 偏差
        
    def forward(self,x):
        return corr2d(x,self.weight.data())+self.bias.data()

In [4]:
# 5.1.3 图像中物体边缘检测 
# 检测图像中物体的边缘，即找到像素变化的位置。
X=nd.ones((6,8))# X代表6x8的图片
X[:,2:6]=0
X


[[1. 1. 0. 0. 0. 0. 1. 1.]
 [1. 1. 0. 0. 0. 0. 1. 1.]
 [1. 1. 0. 0. 0. 0. 1. 1.]
 [1. 1. 0. 0. 0. 0. 1. 1.]
 [1. 1. 0. 0. 0. 0. 1. 1.]
 [1. 1. 0. 0. 0. 0. 1. 1.]]
<NDArray 6x8 @cpu(0)>

In [5]:
# 当卷积核K与输⼊做互相关运算时，如果横向相邻元素相同，输出为0；否则输出为⾮0。 
K=nd.array([[1,-1]])# K.shape为(1,2)
K1=nd.array([1,-1])#  K1,shape为(2,)

In [6]:
Y=corr2d(X,K)
Y# 0和1的边缘检测为1、-1


[[ 0.  1.  0.  0.  0. -1.  0.]
 [ 0.  1.  0.  0.  0. -1.  0.]
 [ 0.  1.  0.  0.  0. -1.  0.]
 [ 0.  1.  0.  0.  0. -1.  0.]
 [ 0.  1.  0.  0.  0. -1.  0.]
 [ 0.  1.  0.  0.  0. -1.  0.]]
<NDArray 6x7 @cpu(0)>

In [12]:
# 5.1.4 通过数据学习核数组
# 构造⼀个输出通道数为1，核数组形状是(1, 2)的二维卷积层 
# Conv2D的第一个参数channels为输出通道数，输入通道数为in_channels默认为0，in_channels如果未指定，
# 卷积层模型参数的初始化将推迟到第一次调用“ forward”时进行并且in_channels将根据输入数据的形状进行推断。
conv2d=nn.Conv2D(1,kernel_size=(1,2))
conv2d.initialize()
print(conv2d.collect_params())

# ⼆维卷积层使⽤4维输⼊输出，格式为(样本, 通道, ⾼, 宽)，这⾥批量⼤⼩（批量中的样本数）和通道数均为1 
X=X.reshape((1,1,6,8))
Y=Y.reshape((1,1,6,7))

for i in range(10):
    with autograd.record():
        Y_hat=conv2d(X)
        l=(Y_hat-Y)**2
    l.backward()
    # 简单起⻅，这⾥忽略了偏差
    conv2d.weight.data()[:] -=3e-2 *conv2d.weight.grad()
    if(i+1)%2 == 0:
        print('batch %d,loss %.3f' %(i+1,l.sum().asscalar()))
        
conv2d.collect_params()

conv1_ (
  Parameter conv1_weight (shape=(1, 0, 1, 2), dtype=<class 'numpy.float32'>)
  Parameter conv1_bias (shape=(1,), dtype=<class 'numpy.float32'>)
)
batch 2,loss 5.063
batch 4,loss 0.864
batch 6,loss 0.151
batch 8,loss 0.028
batch 10,loss 0.006


conv1_ (
  Parameter conv1_weight (shape=(1, 1, 1, 2), dtype=<class 'numpy.float32'>)
  Parameter conv1_bias (shape=(1,), dtype=<class 'numpy.float32'>)
)

In [8]:
conv2d.weight.data().reshape((1,2))


[[ 0.9895    -0.9873705]]
<NDArray 1x2 @cpu(0)>

In [9]:
# 练习1.构造⼀个输⼊图像X，令它有⽔平⽅向的边缘。如何设计卷积核K来检测图像中⽔平边缘？ 如果是对⻆⽅向的边缘呢？ 
# 水平方向的边缘
X1=nd.ones((6,8))
X1[2:4,:]=0
K=nd.array([1,-1]).reshape(2,1)
Y1=corr2d(X1,K)
X1,Y1

# 对角方向的边缘
X1=nd.ones((6,8))
for i in range(min(X1.shape)):
    X1[i,i]=0
K=nd.array([1,1,-1,-1]).reshape(2,2)
Y1=corr2d(X1,K)
X1,Y1

(
 [[0. 1. 1. 1. 1. 1. 1. 1.]
  [1. 0. 1. 1. 1. 1. 1. 1.]
  [1. 1. 0. 1. 1. 1. 1. 1.]
  [1. 1. 1. 0. 1. 1. 1. 1.]
  [1. 1. 1. 1. 0. 1. 1. 1.]
  [1. 1. 1. 1. 1. 0. 1. 1.]]
 <NDArray 6x8 @cpu(0)>,
 
 [[ 0.  1.  0.  0.  0.  0.  0.]
  [-1.  0.  1.  0.  0.  0.  0.]
  [ 0. -1.  0.  1.  0.  0.  0.]
  [ 0.  0. -1.  0.  1.  0.  0.]
  [ 0.  0.  0. -1.  0.  1.  0.]]
 <NDArray 5x7 @cpu(0)>)

In [10]:
import mxnet 
mxnet.gluon??

Object `mxnet.gluon` not found.


In [ ]:
mxnet.gluon.parameter.Parameter??

In [ ]:
mxnet.gluon.parameter??

In [ ]:
mxnet.gluon.parameter.ParameterDict??